# Simulate ALOHA with Script (within Jupyter)

## Introduction

This notebook is (partially) based on:
https://github.com/huggingface/lerobot/blob/main/examples/4_train_policy_with_script.md

## Preparation

In [1]:
import sys

from IPython.display import Video

from lerobot.common.utils.utils import init_logging
from lerobot.configs import parser

from lerobot.configs.eval import EvalPipelineConfig
from lerobot.scripts.eval import eval_main

from lerobot.configs.train import TrainPipelineConfig
from lerobot.scripts.train import train as train_main

In [2]:
init_logging()

## Evaluation

In [3]:
# HACK: Populate the config object by modifying sys.argv
@parser.wrap()
def load_config(config: EvalPipelineConfig):
    return config


sys.argv = sys.argv[:1] + [
    "--policy.path=lerobot/act_aloha_sim_insertion_human",
    "--policy.device=mps",
    "--env.type=aloha",
    "--env.task=AlohaInsertion-v0",
    "--env.episode_length=500",
    "--eval.n_episodes=1",
    "--eval.batch_size=1",
]

config = load_config()

In [4]:
eval_main(config)

INFO 2025-03-09 22:57:59 pts/eval.py:458 {'env': {'episode_length': 500,
         'features': {'action': {'shape': (14,),
                                 'type': <FeatureType.ACTION: 'ACTION'>},
                      'agent_pos': {'shape': (14,),
                                    'type': <FeatureType.STATE: 'STATE'>},
                      'pixels/top': {'shape': (480, 640, 3),
                                     'type': <FeatureType.VISUAL: 'VISUAL'>}},
         'features_map': {'action': 'action',
                          'agent_pos': 'observation.state',
                          'pixels/top': 'observation.images.top',
                          'top': 'observation.image.top'},
         'fps': 50,
         'obs_type': 'pixels_agent_pos',
         'render_mode': 'rgb_array',
         'task': 'AlohaInsertion-v0'},
 'eval': {'batch_size': 1, 'n_episodes': 1, 'use_async_envs': False},
 'job_name': 'aloha_act',
 'output_dir': PosixPath('outputs/eval/2025-03-09/22-57-59_aloha_act'),
 

{'avg_sum_reward': 731.0, 'avg_max_reward': 3.0, 'pc_success': 0.0, 'eval_s': 12.721962213516235, 'eval_ep_s': 12.721965074539185}


In [5]:
Video(config.output_dir / "videos" / "eval_episode_0.mp4")

## Fine-Tuning

In [6]:
# HACK: Populate the config object by modifying sys.argv
@parser.wrap()
def load_config(config: TrainPipelineConfig):
    return config


sys.argv = sys.argv[:1] + [
    "--policy.path=lerobot/act_aloha_sim_transfer_cube_human",
    "--policy.device=mps",
    "--dataset.repo_id=lerobot/aloha_sim_insertion_human",
    "--env.type=aloha",
    "--env.task=AlohaInsertion-v0",
    "--wandb.enable=true", # `uv run wandb login` is required
]

config = load_config()

In [7]:
train_main(config)

INFO 2025-03-09 22:58:14 ts/train.py:111 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

Logs will be synced with wandb.


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

INFO 2025-03-09 22:58:20 ts/train.py:135 Creating env
INFO 2025-03-09 22:58:22 ts/train.py:138 Creating policy
INFO 2025-03-09 22:58:23 ts/train.py:144 Creating optimizer and scheduler
INFO 2025-03-09 22:58:23 ts/train.py:156 Output dir: outputs/train/2025-03-09/22-58-14_aloha_act
INFO 2025-03-09 22:58:23 ts/train.py:158 cfg.env.task='AlohaInsertion-v0'
INFO 2025-03-09 22:58:23 ts/train.py:159 cfg.steps=100000 (100K)
INFO 2025-03-09 22:58:23 ts/train.py:160 dataset.num_frames=25000 (25K)
INFO 2025-03-09 22:58:23 ts/train.py:161 dataset.num_episodes=50
INFO 2025-03-09 22:58:23 ts/train.py:162 num_learnable_params=51613582 (52M)
INFO 2025-03-09 22:58:23 ts/train.py:163 num_total_params=51613672 (52M)
INFO 2025-03-09 22:58:23 ts/train.py:202 Start offline training on a fixed dataset
INFO 2025-03-09 22:59:56 ts/train.py:232 step:200 smpl:2K ep:3 epch:0.06 loss:0.936 grdn:7.090 lr:1.0e-05 updt_s:0.437 data_s:0.030
INFO 2025-03-09 23:01:23 ts/train.py:232 step:400 smpl:3K ep:6 epch:0.13 loss

In [9]:
trained = config.output_dir / "checkpoints" / "last" / "pretrained_model"

In [10]:
# HACK: Populate the config object by modifying sys.argv
@parser.wrap()
def load_config(config: EvalPipelineConfig):
    return config


sys.argv = sys.argv[:1] + [
    f"--policy.path={trained}",
    "--policy.device=mps",
    "--env.type=aloha",
    "--env.task=AlohaInsertion-v0",
    "--env.episode_length=500",
    "--eval.n_episodes=1",
    "--eval.batch_size=1",
]

config_eval = load_config()

In [14]:
eval_main(config_eval)

INFO 2025-03-10 18:36:36 pts/eval.py:458 {'env': {'episode_length': 500,
         'features': {'action': {'shape': (14,),
                                 'type': <FeatureType.ACTION: 'ACTION'>},
                      'agent_pos': {'shape': (14,),
                                    'type': <FeatureType.STATE: 'STATE'>},
                      'pixels/top': {'shape': (480, 640, 3),
                                     'type': <FeatureType.VISUAL: 'VISUAL'>}},
         'features_map': {'action': 'action',
                          'agent_pos': 'observation.state',
                          'pixels/top': 'observation.images.top',
                          'top': 'observation.image.top'},
         'fps': 50,
         'obs_type': 'pixels_agent_pos',
         'render_mode': 'rgb_array',
         'task': 'AlohaInsertion-v0'},
 'eval': {'batch_size': 1, 'n_episodes': 1, 'use_async_envs': False},
 'job_name': 'aloha_act',
 'output_dir': PosixPath('outputs/eval/2025-03-10/18-35-43_aloha_act'),
 

Loading weights from local directory


Stepping through eval batches: 100%|██████████| 1/1 [00:14<00:00, 14.08s/it, running_success_rate=0.0%]
INFO 2025-03-10 18:36:51 pts/eval.py:497 End of eval


{'avg_sum_reward': 0.0, 'avg_max_reward': 0.0, 'pc_success': 0.0, 'eval_s': 14.421576976776123, 'eval_ep_s': 14.421579122543335}


In [15]:
Video(config_eval.output_dir / "videos" / "eval_episode_0.mp4")